In [1]:
# Import required Libraries
import pandas as pd
import os
from pandas import json_normalize
import json
import requests
from IPython.display import JSON

### Import Test Sample

In [107]:
flight_sample = pd.read_csv('./Data/Flight_test_sample.csv')

### Weather Functions

In [3]:
# Weather API Fetch Function
def world_weather_fetch(query):
    
        # Set Key
        weather_key = os.environ['weather_api_key']
        
        # Set Parameters for API
        weather_url = "http://api.worldweatheronline.com/premium/v1/weather.ashx?"
        location = query
        weather_response = requests.get(weather_url,params={
                                                  'q':location,
                                                  'date':'2020-01-01',
                                                  'fx':'no',
                                                  'cc':'no',
                                                  'mca':'yes',
                                                  'format':'json',
                                                  'key':weather_key})
        
        # Call API and direct returned information
        weather_result = weather_response.json()
        df_weather = json_normalize(weather_result)
        return weather_result, df_weather

In [108]:
flight_sample = flight_sample.drop(columns=['avgThunderDays','avgRainDays','avgDryDays','avgCloud','avgSnowDays'])

In [113]:
flight_sample.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'crs_dep_time', 'crs_arr_time', 'dup',
       'crs_elapsed_time', 'flights', 'distance', 'flight_speed',
       'flight_haul_type', 'flight_month', 'arrival_hour_of_day',
       'departure_hour_of_day', 'percent_delay', 'percent_flights',
       'avg_flights', 'avg_passengers', 'total_distance', 'total_passengers',
       'avg_fuel', 'state', 'dest_city_name', 'avgDryDays', 'avgRainDays',
       'avgCloud', 'avgSnowDays'],
      dtype='object')

In [49]:
locations = list(flight_sample.dest_city_name.unique())

In [82]:
def compile_weather_data(string_list):
    
    columns=['avgRainDays','avgDryDays','avgCloud','avgSnowDays']
    
    # Call Api for 1st entry in list and establish dataframe to build upon
    init_result,init_data = world_weather_fetch(string_list[0])
    init_data = json_normalize(data=init_data['data.ClimateAverages'][0][0]['month'][0])
    # Modify dataframe types for specified columns
    init_data = init_data[columns].apply(pd.to_numeric)
    # Add location data to initial dataframe 
    init_data['location'] = string_list[0]
    
    for city in string_list[1:]:
        print(city)
        #Call weather API per location and get dataframe values
        result,data = world_weather_fetch(city)
        data = json_normalize(data=data['data.ClimateAverages'][0][0]['month'][0])
        # Modify dataframe types for specified columns
        data = data[columns].apply(pd.to_numeric)
        # Add location data to dataframe 
        data['location'] = city
        # Append Dataframe to initial dataframe
        init_data = init_data.append(data)
  
    return init_data

### Get Weather Data for Each Destination Airport

In [84]:
weather_data = compile_weather_data(locations)

Sacramento, CA
   avgRainDays  avgDryDays   avgCloud  avgSnowDays        location
0            9          22  33.950638            0  Sacramento, CA
Phoenix, AZ
   avgRainDays  avgDryDays   avgCloud  avgSnowDays     location
0            4          26  18.274073            0  Phoenix, AZ
Denver, CO
   avgRainDays  avgDryDays  avgCloud  avgSnowDays    location
0            4          23  22.60027            6  Denver, CO
San Jose, CA
   avgRainDays  avgDryDays   avgCloud  avgSnowDays      location
0            9          21  30.866936            0  San Jose, CA
Las Vegas, NV
   avgRainDays  avgDryDays   avgCloud  avgSnowDays       location
0            4          26  19.019089            0  Las Vegas, NV
Dallas, TX
   avgRainDays  avgDryDays   avgCloud  avgSnowDays    location
0            8          21  33.223583            0  Dallas, TX
Portland, OR
   avgRainDays  avgDryDays   avgCloud  avgSnowDays      location
0           18          11  57.105152            2  Portland, OR
Chicago

In [87]:
weather_data = weather_data[['location','avgDryDays','avgRainDays','avgCloud','avgSnowDays']]
weather_data = weather_data.rename(columns={"location": "dest_city_name"})

In [110]:
flight_sample = flight_sample.drop(columns=['dest_city_name'])
flight_sample = pd.merge(flight_sample,weather_data,how='outer',left_index=True,right_index=True)

In [115]:

import pickle
outfile = './Data/Flight_2020_Test_sample'
weather_2020_test = open(outfile,'wb')
pickle.dump(flight_sample,weather_2020_test)
weather_2020_test.close()